In [47]:
#!/usr/bin/env python3
import math
import numpy as np
import warnings
import sys
#from scipy.spatial.transform import Rotation
import os 
import general_robotics_toolbox as rox
import time
from new_Arm_Lib import Arm_Device
import task1
import random

# Get DOFBOT object
Arm = Arm_Device()
time.sleep(.1)

def Dofbot():
    #inital the dofbot parameters
    
    x=np.array([1,0,0])
    y=np.array([0,1,0])
    z=np.array([0,0,1])
    a=np.array([0,0,0])
    
    H = np.array([z,-y,-y,-y,-x]).T    
    P = np.array([0.1045*z, a, 0.08285*x, -0.08285*z, a, -0.12842*x]).T
    joint_type=[0,0,0,0,0]

    
    return rox.Robot(H, P, joint_type)    


def inverse_K(Rd, Pd, q0, alpha,  tol=[0.02,0.02,0.02,0.001,0.001,0.001], Nmax=200):
    n = len(q0)
    q = np.zeros(n)
    q = q0
    P0t = np.zeros(3)
    iternum = 0
    fwd = rox.fwdkin(Dofbot(),q0)#do the forward kinematics
    R = fwd.R
    P = fwd.p
    dR = R @ np.transpose(Rd)
    k = np.transpose(rox.R2rpy(dR))  
    dX =  np.concatenate((k, (P-Pd)),axis = None)
    while np.prod(abs(dX)).all()>0.02:
            #check maximum internum
        if iternum <= Nmax:
            fwd = rox.fwdkin(Dofbot(),q)
            R = fwd.R
            P = fwd.p
            #get jacobian matrix
            Jq = rox.robotjacobian(Dofbot(),q)
            dR = R @ np.transpose(Rd)
            k = np.transpose(rox.R2rpy(dR))
            dX =  np.concatenate((k, (P-Pd)),axis = None)
            #update the q
            q=q-alpha*np.linalg.pinv(Jq)@dX
            iternum = iternum+1
            #print(q)
        else:
            break
    return q, P0t, iternum

def path_angle_generate(x,y,z):
    #input values
    Rd = [[-0.75, -0.1047, -0.6531],
          [-0.433, 0.8241, 0.3652],
          [0.5   , 0.5567, -0.6634]]
    Pd = [x,
          y,
          z]
    q0 = np.deg2rad(np.transpose([25, 50, 75, 30, 30]))

    a,b,c = inverse_K(Rd, Pd, q0, alpha=0.3,  tol=[0.02,0.02,0.02,0.001,0.001,0.001], Nmax=200)
    sa = np.rad2deg(a)
    return sa

########################################################################
# PID control
########################################################################
# convert speed from degree/second to millisecond
def speed_conversion(speed, start_angle, desired_angle):
    step = abs(desired_angle - start_angle)
    time = step/speed
    milli_time = round(time * 1000)
    time_sec = round(time)
    return milli_time, time_sec

#def PID_control(kp, kd, ki)


def calculate_pos_error(des_pos):
    # calculate error
    ca = [0]*5
    for i in range(0,5):
        ca[i] = Arm.Arm_serial_servo_read(i+1)
        time.sleep(0.01)
        ca[i] = float(ca[i]) * math.pi / 180
    
    current_pos = task1.position(ca[0],ca[1],ca[2],ca[3],ca[4])

    error = []
    for i in range(0,3):
        error.append(des_pos[i]-current_pos[i])
        
    return error



if __name__ == "__main__":
    #input values
    # x start = 0.0558 end = 0.2058
    # calculate each step
    split_time = 20
    start = 0.0558
    end = 0.2058
    step = float((end-start)/split_time)
    
    '''
    # reach the start position
    angle = path_angle_generate(start, 0.1188, 0.1464)
    Arm.Arm_serial_servo_write6(angle[0], angle[1], angle[2], angle[3], angle[4], 170, 500)
    time.sleep(1)
    '''
    
    # Enter the desired postion (q_desired)
    desired_pos = [0.2058, 0.1188, 0.1464]
    start_pos = [0.0558, 0.1188, 0.1464]
    
    # Print calculated final servo angle
    print("calculated final servo angle: ",path_angle_generate(desired_pos[0], desired_pos[1], desired_pos[2]))
    
    
    # Initial the PID value
    qmeasured = [0]*3
    error = [0]*3
    error_prev = [0]*3
    error_sum = [0]*3
    pid_pos = 0
    pid_speed_pos = 0
    
    # initial PID
    KP = 0.02
    KD = 0.01
    KI = 0.005
    
    # Move the servo in 5 steps,
    # PID control included
    for i in range(0,split_time):
        
        #####################################################################################
        #################################### PID ############################################
        #####################################################################################
        # Include PID control to modify the position for each step (start variable)

        # Calculate the position error
        error = calculate_pos_error(start_pos)

        # calculate pid position for each position
        postion_start_pid = [0]*3
        postion_start_pid[0] = error[0] * KP + error_prev[0] * KD + error_sum[0] * KI
        postion_start_pid[1] = error[1] * KP + error_prev[1] * KD + error_sum[1] * KI
        postion_start_pid[2] = error[2] * KP + error_prev[2] * KD + error_sum[2] * KI
        
        
        # ----------- move
        # move the servo by the calculated position
        # Calculate the servo angle for reaching the disired position
        angle = path_angle_generate(start_pos[0], start_pos[1], start_pos[2])
        
        # Move the servo to the desired position
        Arm.Arm_serial_servo_write6(angle[0], angle[1], angle[2], angle[3], angle[4], 170, 500)
        time.sleep(0.5)
        #------------ move 
        out_ragne = False
        for a in angle: 
            if a > 180:
                out_ragne = True
                
        if not out_ragne: 
            # Calculate the postion for next step
            start_pos[0] = float(start_pos[0] + step + postion_start_pid[0])
            start_pos[1] = float(start_pos[1] + postion_start_pid[1])
            start_pos[2] = float(start_pos[2] + postion_start_pid[2])

            # calculate the error
            error_prev = error
            error_sum[0] = abs(error[0] + error_sum[0])
            error_sum[1] = abs(error[1] + error_sum[1])
            error_sum[2] = abs(error[2] + error_sum[2])
        else:
            start_pos[0] = float(start_pos[0])
        
        #####################################################################################


    error = calculate_pos_error(desired_pos)
    print("Error: ", error[0], error[1], error[2])

        
    

calculated final servo angle:  [30.00087374 45.11811687 79.78275984 25.10027228 40.00109383]
参数传入范围不在0-180之内！The parameter input range is not within 0-180!
参数传入范围不在0-180之内！The parameter input range is not within 0-180!
参数传入范围不在0-180之内！The parameter input range is not within 0-180!
参数传入范围不在0-180之内！The parameter input range is not within 0-180!
Error:  [0.01310436] [0.00409602] [0.00087923]
